# PyTorch Differential Privacy Experiment

In [ ]:
%pip install -qqq torch torchvision opacus numpy pandas
%pip install -qqq wandb datasets tqdm


In [ ]:
import os
import torch 
from dotenv import load_dotenv
import wandb 
import logging
import shutil
import sys
from datetime import datetime, timedelta

import argparse
from collections import Counter
from pathlib import Path
from statistics import mean

import torch
import torch.nn as nn
from opacus import PrivacyEngine
from opacus.layers import DPGRU, DPLSTM, DPRNN
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, tqdm_notebook

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
if os.path.exists('.env'):
    load_dotenv('.env')
device


In [ ]:
logging.basicConfig(
    format="%(asctime)s:%(levelname)s:%(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    stream=sys.stdout,
)
logger = logging.getLogger("ddp")
logger.setLevel(level=logging.INFO)


In [ ]:
wandb.login(key=os.getenv('WANDB_API_KEY'))
wandb.init(project="verida-pii", name="deberta_finetune")

# Fine Tuning w/ Unsloth (Colab Only)

In [ ]:
# Datasets
# Original data_name = 'Ezi/medical_and_legislators_synthetic'
# Tutorial: https://huggingface.co/blog/Andyrasika/finetune-unsloth-qlora


In [ ]:
# Get the major and minor version of the current CUDA device (GPU)
major_version, minor_version = torch.cuda.get_device_capability()

# Apply the following if the GPU has Ampere or Hopper architecture (RTX 30xx, RTX 40xx, A100, H100, L40, etc.)
if major_version >= 8:
    # Install the Unsloth library for Ampere and Hopper architecture from GitHub
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git" -q

# Apply the following for older GPUs (V100, Tesla T4, RTX 20xx, etc.)
else:
    # Install the Unsloth library for older GPUs from GitHub
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" -q

# Placeholder statement (does nothing)
pass

# Install the Hugging Face Transformers library from GitHub, which allows native 4-bit loading
!pip install "git+https://github.com/huggingface/transformers.git" -q



In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("lakshyakh93/deberta_finetuned_pii")
model = AutoModelForTokenClassification.from_pretrained("lakshyakh93/deberta_finetuned_pii")

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    # Specify the existing model

    r=16,  # Choose any positive number! Recommended values include 8, 16, 32, 64, 128, etc.
    # Rank parameter for LoRA. The smaller this value, the fewer parameters will be modified.

    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
    # Specify the modules to which LoRA will be applied

    lora_alpha=16,
    # Alpha parameter for LoRA. This value determines the strength of the applied LoRA.

    lora_dropout=0,  # Currently, only supports dropout = 0
    # Dropout rate for LoRA. Currently supports only 0.

    bias="none",  # Currently, only supports bias = "none"
    # Bias usage setting. Currently supports only the setting without bias.

    use_gradient_checkpointing=True,
    # Whether to use gradient checkpointing to improve memory efficiency

    random_state=3407,
    # Seed value for random number generation

    max_seq_length=max_seq_length,
    # Set the maximum sequence length
)

In [ ]:
# @TODO - Add the relevant prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
# Define the prompt format for the Alpaca dataset

def formatting_prompts_func(examples):
    # Define a function to format each example in the dataset

    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    # Get instructions, inputs, and outputs

    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Generate text by combining instructions, inputs, and outputs

        text = alpaca_prompt.format(instruction, input, output)
        # Format the text according to the prompt format

        texts.append(text)
    return { "text" : texts, }
    # Return a list of formatted texts

pass
# Placeholder (does nothing)

from datasets import load_dataset
# Import the load_dataset function from the datasets library

dataset = load_dataset("yahma/alpaca-cleaned", split="train")
# Load the training data of the cleaned version of the Alpaca dataset from yahma

dataset = dataset.map(formatting_prompts_func, batched=True,)

In [ ]:
from trl import SFTTrainer
# Import SFTTrainer from the TRL library

from transformers import TrainingArguments
# Import TrainingArguments from the Transformers library

trainer = SFTTrainer(
    # Initialize the SFTTrainer

    model=model,
    # Specify the model to be used

    train_dataset=dataset,
    # Specify the training dataset

    dataset_text_field="text",
    # Specify the text field in the dataset

    max_seq_length=max_seq_length,
    # Specify the maximum sequence length

    args=TrainingArguments(
        # Specify training arguments

        per_device_train_batch_size=2,
        # Specify the training batch size per device

        gradient_accumulation_steps=4,
        # Specify the number of steps for gradient accumulation

        warmup_steps=5,
        # Specify the number of warm-up steps

        max_steps=20,
        # Specify the maximum number of steps

        learning_rate=2e-4,
        # Specify the learning rate

        fp16=not torch.cuda.is_bf16_supported(),
        # Set whether to use 16-bit floating-point precision (fp16)

        bf16=torch.cuda.is_bf16_supported(),
        # Set whether to use Bfloat16

        logging_steps=1,
        # Specify the logging steps

        optim="adamw_8bit",
        # Specify the optimizer (here using 8-bit AdamW)

        weight_decay=0.01,
        # Specify the weight decay value

        lr_scheduler_type="linear",
        # Specify the type of learning rate scheduler (linear)

        seed=3407,
        # Specify the random seed

        output_dir="outputs",
        # Specify the output directory

    ),
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
# Get properties of the GPU device at index 0

start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# Get the maximum reserved GPU memory in GB and round to 3 decimal places

max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
# Get the total GPU memory in GB and round to 3 decimal places

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
# Display the GPU name and maximum memory

print(f"{start_gpu_memory} GB of memory reserved.")
# Display the reserved memory amount

In [ ]:
trainer_stats = trainer.train()

# Convert to GGUF

In [ ]:
def colab_quantize_to_gguf(save_directory, quantization_method="q4_k_m"):
    # Define a function for conversion to GGUF

    from transformers.models.llama.modeling_llama import logger
    import os
    # Import necessary libraries

    logger.warning_once(
        "Unsloth: `colab_quantize_to_gguf` is still in development mode.\n"\
        "If anything errors or breaks, please file a ticket on Github.\n"\
        "Also, if you used this successfully, please tell us on Discord!"
    )
    # Warn that it's still in development mode and encourage reporting any issues

    # From https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html
    ALLOWED_QUANTS = \
    {
        # Define currently allowed quantization methods
        # Including descriptions for each quantization method
        "q2_k"   : "Uses Q4_K for the attention.vw and feed_forward.w2 tensors, Q2_K for the other tensors.",
        "q3_k_l" : "Uses Q5_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K",
        "q3_k_m" : "Uses Q4_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K",
        "q3_k_s" : "Uses Q3_K for all tensors",
        "q4_0"   : "Original quant method, 4-bit.",
        "q4_1"   : "Higher accuracy than q4_0 but not as high as q5_0. However has quicker inference than q5 models.",
        "q4_k_m" : "Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K",
        "q4_k_s" : "Uses Q4_K for all tensors",
        "q5_0"   : "Higher accuracy, higher resource usage and slower inference.",
        "q5_1"   : "Even higher accuracy, resource usage and slower inference.",
        "q5_k_m" : "Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K",
        "q5_k_s" : "Uses Q5_K for all tensors",
        "q6_k"   : "Uses Q8_K for all tensors",
        "q8_0"   : "Almost indistinguishable from float16. High resource use and slow. Not recommended for most users.",
    }

    if quantization_method not in ALLOWED_QUANTS.keys():
        # If the specified quantization method is not allowed, raise an error
        error = f"Unsloth: Quant method = [{quantization_method}] not supported. Choose from below:\n"
        for key, value in ALLOWED_QUANTS.items():
            error += f"[{key}] => {value}\n"
        raise RuntimeError(error)

    # Display information about the conversion
    print_info = \
        f"==((====))==  Unsloth: Conversion from QLoRA to GGUF information\n"\
        f"   \\\   /|    [0] Installing llama.cpp will take 3 minutes.\n"\
        f"O^O/ \_/ \\    [1] Converting HF to GUUF 16bits will take 3 minutes.\n"\
        f"\        /    [2] Converting GGUF 16bits to q4_k_m will take 20 minutes.\n"\
        f' "-____-"     In total, you will have to wait around 26 minutes.\n'
    print(print_info)
    # Display information about the conversion process

    if not os.path.exists("llama.cpp"):
        # If llama.cpp does not exist, install it
        print("Unsloth: [0] Installing llama.cpp. This will take 3 minutes...")
        !git clone https://github.com/ggerganov/llama.cpp
        !cd llama.cpp && make clean && LLAMA_CUBLAS=1 make -j
        !pip install gguf protobuf
        pass

    print("Unsloth: Starting conversion from HF to GGUF 16bit...")
    # Display that conversion from HF to GGUF 16bit is starting
    # print("Unsloth: [1] Converting HF into GGUF 16bit. This will take 3 minutes...")
    !python llama.cpp/convert.py {save_directory} \
        --outfile {save_directory}-unsloth.gguf \
        --outtype f16

    print("Unsloth: Starting conversion from GGUF 16bit to q4_k_m...")
    # Display that conversion from GGUF 16bit to the specified quantization method is starting
    # print("Unsloth: [2] Converting GGUF 16bit into q4_k_m. This will take 20 minutes...")
    final_location = f"./{save_directory}-{quantization_method}-unsloth.gguf"
    !./llama.cpp/quantize ./{save_directory}-unsloth.gguf \
        {final_location} {quantization_method}

    print(f"Unsloth: Output location: {final_location}")
    # Display the output location of the converted file
pass

In [ ]:
from unsloth import unsloth_save_model
# Import the unsloth_save_model function from the Unsloth library

# unsloth_save_model has the same args as model.save_pretrained
# unsloth_save_model has the same arguments as model.save_pretrained
unsloth_save_model(model, tokenizer, "output_model", push_to_hub=False, token=None)
# Save the model and tokenizer as "output_model". Do not push to the Hugging Face Hub

colab_quantize_to_gguf("output_model", quantization_method="q4_k_m")
# Convert "output_model" to GGUF format. Use the quantization method "q4_k_m"